# Computing velocities from positions

If you run this notebook on your own computer (recommended), you need:
* julia (https://julialang.org/downloads/#official_binaries_for_manual_download)
* some julia packages:
    * `PythonPlot` (or `PyPlot`, but `PyPlot` is difficult to install on Windows)
    * `DelimitedFiles` for reading CSV files
    * `DSP` for digital signal processing)
    * `NCDatasets` for loading NetCDF files

* **If** you are using your own computer, you can install these packages with the following julia commands:

```julia
using Pkg
Pkg.add(["PythonPlot", "DelimitedFiles", "DSP", "NCDatasets"])
using PythonPlot, DelimitedFiles, DSP, NCDatasets
```

But the packages need to be installed before the lecture.
The steps above are not necessary if you work on the notebook server (https://notebook-gher.uliege.be), **but you need to download your files after each lesson (as they are deleted)**. 

## Exercice

* Compute the drifter velocities from the provided file (drifter number 1).
* Remove the part of the data where the drifter was not deployed or when it was stuck at the coast
* Smooth the data with a second order Butterworth filter (with a cut-off frequency corresponding to a period of 30 minutes) from the `DSP` package (example code is provided below).
* Plot the trajectory and the currents with the coastline and bathymetry (isobath 25 m, 50 m, 75 m, 100 m)
  
Data files:
* Drifter positions: https://dox.ulg.ac.be/index.php/s/A3KkDQYmdNLlabx/download . The data is in the CSV format (comma-separated values) where the columns are the time (in UTC), longitude (degree East) and latitude (degree North).
* Bathymetry: https://dox.ulg.ac.be/index.php/s/9ZwWDbsTgCwgS90/download as a NetCDF file.

You are explicitly encouraged to consult the documentation of these software packages and to use a search engine.

This is the metadata for the deployement from previous students:

```
Timing drifters le 08 mai: 
1: 9h12-14h31 
2: 9h14-14h37 
3: 9h07-14h12 
4: 9:11-14h26 
5: 9h10-14h20 
6: 9h13-14h32 
7: 9h14-14h34 
8: 9h08-14h12 
9: 9h06-14h18 
10: 9h09-14h24
```

The year is 2024 and the times are in local time (UTC-2). The number in the first column is the drifter number.

Additional information:
* The module `DelimitedFiles` allows you to read CSV formatted files with the command `readdlm`. The command `?readdlm` provides for more information. Use for example `readdlm(filename,',');` of the columns a separated by a comma.
* You can convert a string `"2024-05-08T06:20:15"` into a `DateTime` structure (from the module `Dates`) by calling `DateTime("2024-05-08T06:20:15")`
* You can compute the number of seconds between e.g. `"2024-05-08T06:23:34"` and `"2024-05-08T06:20:15"` using:
```julia
using Dates
t0 = DateTime("2024-05-08T06:20:15")
t1 = DateTime("2024-05-08T06:23:34")
time_difference = Dates.value(t1 - t0)/1000 # convert milliseconds to seconds
```
* The function `plot(lon,lat)` from the module `PythonPlot` allows you to plot the trajectory given by the longitudes `lon` and latitudes `lat`.

* You can assume that the Earth is a sphere with a radius of 6371009 m. You will need the Earth Radius to convert an increment in degrees to meters.
* The package DSP allow you to apply a 2nd order Butterworth filter. Here is an example using random date `x`. The filtered data is `xf`.

In [ ]:
using DSP, PythonPlot
# Filter parameters
fs = 1/60 # sampling frequency (Hz)
# Cut-off frequency (Hz), 30 minutes
fc = 1/(30*60)

# 2nd order Butterworth filter
bfilter = digitalfilter(Lowpass(fc, fs=fs), Butterworth(2))
x = randn(200);
xf = filtfilt(bfilter, x);
plot(x,label="original data")
plot(xf,label="filtered data")
legend()

* To download and load the bathymetric data you can use the following code snipped:

In [ ]:
using NCDatasets
using Downloads: download
bathname = "nwcorsica_bathymetry.nc";

if !isfile(bathname)
    download("https://dox.ulg.ac.be/index.php/s/9ZwWDbsTgCwgS90/download",bathname)
end
ds = NCDataset(bathname);
# select only the data within a given bounding box
dss = NCDatasets.@select(ds, 8.7 <= lon <= 8.75 && 42.554 <= lat <= 42.6039)
bat_lon = dss["lon"][:];
bat_lat = dss["lat"][:];
bat = nomissing(dss["bat"][:,:],NaN);

* Here some useful plotting functions:
    * The function `pcolor` (for example `pcolor(bat_lon,bat_lat,bat');`) can be used to visualize the bathymetry
    * The function `contour` (for example `contour(bat_lon,bat_lat,-bat',levels = [0],colors="k",linewidths=2)`) can be used to draw the coastlines or `contour(bat_lon,bat_lat,-bat',levels = [25, 50, 75, 100],colors="k",linewidths=0.2); ` to draw the isobaths 25 m, 50 m, 75 m and 100 m 
    * The function `quiver` (for example `q = quiver(lonf,latf,uf,vf,color="b");`) can be used to draw vectors. The function `quiverkey` adds a legend for quiver (for example `quiverkey(q,0.2,0.9,0.1,label = "0.1 m/s")`)


In [ ]:
using PythonPlot
using Dates
using DelimitedFiles
using Statistics
using DSP # DSP version 0.7.9
using Downloads: download

Download an example CSV file:

In [ ]:
filename = "Drifter01_08-05-2024.txt"

if !isfile(filename)
    download("https://dox.ulg.ac.be/index.php/s/A3KkDQYmdNLlabx/download",filename)
end

The text file is a comma separated value file (CSV) where the columns are longitude, latitude, time. 